# Multimodal Rota Belirleme — Deney Not Defteri

Bu notebook; A*, NSGA-II ve RAPTOR-like algoritmalarının performanslarını
küçük ölçekli yapay Ankara grafında test etmek amacıyla hazırlanmıştır.

Deneyler:
- A* algoritması (kısıtlı, tek amaçlı)
- NSGA-II (çok amaçlı)
- RAPTOR-benzeri toplu taşıma algoritması
- Kısıtlara göre rota değişimi örnekleri
- Başlangıç/hedef çiftlerinin karşılaştırılması


In [ ]:
import networkx as nx
import pandas as pd

from utils import load_default_graph, path_stats
from astar_solver import solve_astar_constrained
from nsga_solver import run_nsga2
from raptor_solver import raptor_like

G = load_default_graph()

print("Düğümler:", len(G.nodes()))
print("Kenarlar:", len(G.edges()))


In [ ]:
start, goal = "N6", "N8"

path, t, c = solve_astar_constrained(
    G, start, goal,
    allowed_modes=["metro", "bus", "train", "walk", "car"],
    max_cost=100,
    max_time=120
)

print("A* Sonucu:", path)
print("Süre:", t, "dk   Maliyet:", c, "TL")


In [ ]:
tests = [
    ("sadece toplu taşıma", ["metro","bus","train"], 120, 100),
    ("sadece yürüme", ["walk"], 120, 100),
    ("araba yok + maliyet < 20", ["metro","bus","train","walk","bike"], 120, 20),
]

for title, modes, max_time, max_cost in tests:
    print("----", title, "----")
    path, t, c = solve_astar_constrained(G, start, goal, modes, max_cost, max_time)
    print("Sonuç:", path, "| Süre:", t, "| Maliyet:", c)


In [ ]:
solutions = run_nsga2(G, start, goal, n_generations=40, pop_size=40)

print("Toplam çözüm:", len(solutions))

df = pd.DataFrame([
    {
        "Rota": " → ".join(s["full_path"]),
        "Süre": s["total_time"],
        "Maliyet": s["total_cost"],
        "Aktarma": s["transfers"],
    }
    for s in solutions
])

df.head(10)


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["Süre"], df["Maliyet"])
plt.xlabel("Süre (dk)")
plt.ylabel("Maliyet (TL)")
plt.title("Pareto Dağılımı")
plt.grid(True)
plt.show()
